<a href="https://colab.research.google.com/github/TomoharuKurosu/Tomoharu_DS2/blob/main/graffer_%E8%A7%A3%E8%AA%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#graffer確認
# %%
from pathlib import Path #Pathlib は、ファイルシステムのパス操作を行うためのモジュールです。Path クラスは、ファイルやディレクトリのパスを扱うオブジェクトを提供し、パス操作を直感的に行うことができます。
from tqdm import tqdm #Tqdm は、プログレスバーを表示するためのライブラリです。ループの進捗状況を視覚的に表示することで、処理がどれくらい進んでいるかを確認できます。
#sys: Pythonのシステムパラメータや関数にアクセスするためのモジュールです。例えば、スクリプトの終了やコマンドライン引数の取得などに使用します。
#os: オペレーティングシステムに依存した機能を提供するモジュールです。ファイルやディレクトリの操作、環境変数の取得などが可能です。
#time: 時間に関連する機能を提供するモジュールです。例えば、処理の待機時間を設ける sleep 関数や現在の時間を取得する関数があります。
#datetime: 日付や時間を扱うためのモジュールです。日時の計算やフォーマットなどに使用します。
import sys, os, time, datetime
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns, japanize_matplotlib
#Matplotlib (plt としてインポート): データの可視化を行うためのライブラリです。折れ線グラフ、ヒストグラム、散布図などの基本的なグラフを作成できます。
#Seaborn (sns としてインポート): Matplotlib を基にした高レベルのデータ可視化ライブラリで、統計グラフの作成を簡単に行うことができます。グラフのスタイルやテーマが簡単に設定でき、複雑なプロットも簡単に作成できます。
#Japanize Matplotlib: Matplotlib で日本語を表示するためのライブラリです。Matplotlib で作成したグラフに日本語フォントを設定し、日本語テキストを美しく表示できるようにします。
pd.options.display.max_rows = 100#データフレームを表示する際に、最大で100行まで表示する設定です。
plt.rcParams['figure.figsize'] = (10, 6)#、図の横幅が10インチ、高さが6インチに設定されています。
plt.rcParams['font.size'] = 14#グラフに使用するフォントのサイズを14ポイントに設定しています。
plt.style.use('ggplot')#ggplot は、R言語の ggplot2 パッケージにインスパイアされたスタイルで、背景がグレーで白いグリッド線が表示されるシンプルで見やすいデザインです。
plt.title('title')#ラフのタイトルを 'title' と設定しています。
plt.plot()#空のプロットを作成しています。
plt.xlabel('label')#x軸のラベルを 'label' と設定しています
plt.ylabel('label')#y軸のラベルを 'label' と設定しています
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
#bbox_to_anchor=(1.05, 1): 凡例の位置をグラフ外の右上に設定しています。
#loc='upper left': 凡例の基準位置を左上に設定しています。
#borderaxespad=0: 凡例とグラフとの間の余白をゼロに設定しています。
#fontsize=18: 凡例のフォントサイズを18ポイントに設定しています。
# %%
def clns(df):
    return (
        df
        .rename(columns={"Unnamed: 0":"date_"})
        .assign(date_ = lambda df: pd.to_datetime(df["date_"], unit='D', origin='1899-12-30'))
    )
sheets_ = (
    pd
    .read_excel("../data_raw/【和泉市様】8月～1月の手続きガイド日別レポート.xlsx", skiprows=1, sheet_name=None)
)
#assign メソッドを使って、新しい date_ 列を作成しています。lambda 関数を用いて、date_ 列の値を日付形式に変換しています。
#pd.to_datetime 関数を使用して、日付の形式に変換します。特に、日付がExcelのシリアル日付（1900年からの経過日数）で保存されていることを前提とし、1899-12-30 を基準として日付を計算しています。
#skiprows=1 は、Excelシートの最初の1行をスキップする設定です。
#sheet_name=None は、すべてのシートを辞書形式で読み込む設定です。各シートの名前が辞書のキーになり、そのシートのデータが値として格納されます。
df_all = (
    pd
    .concat([#pd.concat 関数を使って、リスト内の複数のデータフレームを結合し、一つのデータフレームにしています。
        k
        .pipe(clns)#clns 関数は、データフレームの特定の列を日付形式に変換する処理を行います。
        for i,k in sheets_.items()#for i, k in sheets_.items() では、sheets_ 内のすべてのシートをループ処理しています。i にはシート名、k にはそのシートのデータフレームがそれぞれ格納されます。
        if "全ページ" in i#各シート名 (i) に "全ページ" という文字列が含まれているかどうかをチェックします。含まれている場合のみ、そのシートを処理します。
    ])
    .reset_index(drop=True)#既存のインデックスは破棄され、新しいインデックスが振り直されます。
)

df_hitori = (
    pd
    .concat([
        k
        .pipe(clns)
        for i,k in sheets_.items()
        if not "全ページ" in i
    ])
    .set_index("date_")#変換された日付列 date_ をインデックスに設定します。
    .resample("D")#この操作は、指定された期間ごとにデータを集計または再構築するために使用されます。
    .max()#再サンプリングされた各期間内の最大値を計算します。
    .fillna(0)#欠損値 (NaN) を 0 で埋めます。
    .reset_index()
)
# %%
df_all.shape
#データフレームのサイズを確認できます。
# %%
df_hitori.shape#各データフレームのサイズを確認できます。

# %%
(
    plt.figure(figsize=(16,8)),#グラフのサイズを横16インチ、縦8インチに設定します。
    df_all
    .set_index("date_")
    .plot(marker=".")#df_all データフレームの date_ 列をインデックスに設定し、グラフを描画します。marker="." を指定して、各データポイントをドット（点）で表示します。
    , plt.yscale("log")#y軸のスケールを対数スケールに設定します。
    , plt.title("全ページ指標")
)
# %%
(
    plt.figure(figsize=(16,8)),
    df_hitori
    .set_index("date_")
    .plot(marker=".")
    , plt.yscale("log")
    , plt.title("ひとり親家庭支援 指標")
)#df_hitori データフレームを使って、日付をインデックスにした折れ線グラフを描画します。各部分の説明は以下の通りです。
# %%
(
    df_hitori
    .set_index("date_")
    .resample("D")#リサンプリングとは、データを新しい頻度に変換する操作のことです。ここでは、データを日ごとの単位に変換しています。
    .max()
)
# %%
(
    df_all
    .assign(dow = lambda df: df.date_.dt.dayofweek)#date_ 列から曜日を数値（0: 月曜日, 1: 火曜日, ..., 6: 日曜日）として抽出し、新しい dow 列に格納します。
    .assign(dow = lambda df: df.dow.map({
        0:"月",
        1:"火",
        2:"水",
        3:"木",
        4:"金",
        5:"土",
        6:"日",
    }))#dow 列の数値を日本語の曜日名に変換します。例えば、0 は "月" に、1 は "火" に変換されます。
    .pipe(
        sns.barplot
        , x="dow"
        , y="ページビュー数"#曜日別のページビュー数の棒グラフを作成します。
        , errorbar=("se", 1.96)#errorbar=("se", 1.96) は標準誤差に基づく信頼区間を 1.96 倍してエラーバーを表示します（およそ 95% の信頼区間）。
    )
)
# %%
from causalimpact import CausalImpact#介入前後のデータに基づいて因果効果を推定します。
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf#plot_acf: 自己相関関数（ACF）をプロットするための関数です。ACF は、時系列データの各時点の値が過去の値とどの程度相関しているかを示します。これにより、時系列データの季節性や周期性のパターンを確認できます。
#plot_pacf: 偏自己相関関数（PACF）をプロットするための関数です。PACF は、時系列データの各時点の値が過去の値とどの程度独立して相関しているかを示します。PACF は、ARIMA モデルのパラメータを選定する際に役立ちます。
# %%
mrt = (
    df_all
    .set_index("date_")
    .drop(columns="ページビュー数")
    # .assign(前日訪問数 = lambda df: df["訪問数"].shift())
    # .assign(前週訪問数 = lambda df: df["訪問数"].shift(7))
    .assign(月初フラグ = lambda df: (df.index.day == 1) * 1)
    .assign(土日フラグ = lambda df: (df.index.dayofweek >= 5) * 1)#土日フラグ 列を追加します。これは、日付が土曜日または日曜日である場合に 1 を、それ以外の場合に 0 を設定するフラグです。d
    .loc[lambda df: df.index <= "2024-01-01"]
    .dropna()#欠損値を含む行を削除します。これにより、データの品質を保つことができます。
    # .head(10)
)
mrt
# %%
pre_period = ['2023-08-01', '2023-12-25']#pre_period は、介入（またはイベント）が発生する前の期間を示します。ここでは 2023-08-01 から 2023-12-25 までの期間が指定されています。
post_period = ['2023-12-26', '2024-01-01']#post_period は、介入後の評価期間を示します。ここでは 2023-12-26 から 2024-01-01 までの期間が指定されています。
ci = CausalImpact(
    mrt,
    pre_period,
    post_period,
    # estimation="pymc", model_args={"ndraws":3000, "nburn":300},
    # model_args={"nseasons":7},
    # prior_level_sd=0.01,
    model_args={
        # "niter":3000,
        # "standardize":True,
        "fit_method":"hmc",#fit_method は因果推論モデルのフィット方法を指定します。ここでは hmc（ハミルトニアンモンテカルロ法）が指定されています。
        "nseasons":7,#nseasons は季節性の周期を指定します。ここでは 7 が指定されており、1週間（7日）を周期とする季節性を考慮します。
        # "season_duration":1,
        #season_duration=7 と設定した場合:
#1週間（7日）の周期で季節性が繰り返されることを意味します。たとえば、毎週特定の日に訪問者数が増加するようなデータに適用する場合、週ごとのパターンを捉えることができます。
        # "prior_level_sd": 0.01,
        # "dynamic_regression": True,#ダイナミック回帰を有効にするオプションです。ダイナミック回帰では、回帰変数の係数が時間とともに変化することを許容します。
    },
)
ci.run()#指定したデータに基づいて因果推論モデルを実行します。指定された介入前後の期間のデータを使って、介入の因果効果を推定します。
ci.plot(figsize=(22, 20))#figsize 引数を使って図のサイズを (22, 20) インチに設定しています。
ci.summary()
# %%
pre_period = ['2023-08-01', '2023-12-25']
post_period = ['2023-12-26', '2023-12-28']
ci = CausalImpact(
    mrt,
    pre_period,
    post_period,
    # estimation="pymc", model_args={"ndraws":3000, "nburn":300},
    # model_args={"nseasons":7},
    # prior_level_sd=0.01,
    model_args={
        # "niter":3000,
        # "standardize":True,
        "fit_method":"hmc",
        "nseasons":7,
        # "season_duration":1,
        # "prior_level_sd": 0.01,
        # "dynamic_regression": True,
    },
)
ci.run()
ci.plot(figsize=(22, 20))
ci.summary()
# %%
pre_period = ['2023-08-01', '2023-12-25']
post_period = ['2023-12-26', '2023-12-26']
ci = CausalImpact(
    mrt,
    pre_period,
    post_period,
    # estimation="pymc", model_args={"ndraws":3000, "nburn":300},
    # model_args={"nseasons":7},
    # prior_level_sd=0.01,
    model_args={
        # "niter":3000,
        # "standardize":True,
        "fit_method":"hmc",
        "nseasons":7,
        # "season_duration":1,
        # "prior_level_sd": 0.01,
        # "dynamic_regression": True,
    },
)
ci.run()#ci.run() は、設定した事前期間と事後期間に基づいて因果影響分析を実行します。このステップで、モデルがデータに適合され、事後期間における影響が推定されます。
ci.plot(figsize=(22, 20))#ci.plot() は、CausalImpact の結果を可視化します。このプロットは、以下の3つのグラフで構成されることが多いです。
ci.summary()#因果影響分析の結果を要約したテキストを表示します。この要約には、以下の情報が含まれます。



# %%
pre_period = ['2023-08-01', '2023-12-25']
post_period = ['2023-12-26', '2023-12-26']
ci = CausalImpact(
    mrt,
    pre_period,
    post_period,
    # estimation="pymc", model_args={"ndraws":3000, "nburn":300},
    # model_args={"nseasons":7},
    # prior_level_sd=0.01,
    model_args={
        # "niter":3000,
        # "standardize":True,
        "fit_method":"hmc",
        "nseasons":7,
        # "season_duration":1,
        # "prior_level_sd": 0.01,
        # "dynamic_regression": True,
    },
)
ci.run()
ci.plot(figsize=(22, 20))
ci.summary()